# Buildkite API development

In [ ]:
!cd ..

In [ ]:
import dotenv

In [ ]:
dotenv.load_dotenv()

In [ ]:
import os

token = os.getenv("BUILDKITE_PAT")

In [ ]:
import requests

base_url = "https://api.buildkite.com/v2"

h = {"Authorization": f"Bearer {token}"}

In [ ]:
pipelines = requests.get(base_url + "/organizations/clima/pipelines", headers=h).json()

for pl in pipelines:
    print(pl["slug"])

In [ ]:
def get_jobs(created_from: str) -> list[dict]:
    """Get all flagship AMIP jobs since ``created_from``."""
    jobs = []
    builds = requests.get(
        base_url + "/organizations/clima/pipelines/climacoupler-coarse-nightly-amip/builds",
        headers=h,
        params={"created_from": created_from},
    ).json()
    for b in builds:
        for j in b["jobs"]:
            name = j.get("name") or ""
            if "flagship amip" in name.lower() and "16 helem" in name.lower():
                job = {
                    "build_id": b.get("id"),
                    "build_state": b.get("state"),
                    "build_created_at": b.get("created_at"),
                    "commit": b.get("commit"),
                    "branch": b.get("branch"),
                    "name": name,
                    "command": j.get("command"),
                    "state": j.get("state"),
                }
                raw_log_url = j.get("raw_log_url")
                if raw_log_url:
                    log_resp = requests.get(raw_log_url, headers=h)
                    job["raw_log_txt"] = log_resp.text
                jobs.append(job)
    return jobs

jobs = get_jobs("2026-01-10")
print(len(jobs))
jobs[0]

In [ ]:
jobs = get_jobs("2026-01-01")

In [ ]:
len(jobs)

In [ ]:
jobs[0]